In [ ]:
import jupedsim as jps
from shapely import Polygon
import pathlib
import matplotlib.pyplot as plt
import shapely

In [ ]:
with open("geometry.wkt") as f:
    geometry_str = f.readline()

area = shapely.from_wkt(geometry_str)
for geo in area.geoms:
    plt.plot(*geo.exterior.xy)
    for interior in geo.interiors:
        plt.plot(*interior.xy, "-k")

plt.show()

In [ ]:
def distribute_agents_in_polygon(
    x1: float,  
    y1: float,
    x2: float,
    y2: float,
    number_of_agents: int = 3,
    distance_to_agents: float = 0.4,
    distance_to_polygon: float = 0.2,
    seed: int = 1,
):
    polygon = Polygon([[x1, y1], [x2, y1], [x2, y2], [x1, y2]])
    return jps.distributions.distribute_by_number(
        polygon=polygon,
        number_of_agents=number_of_agents,
        distance_to_agents=distance_to_agents,
        distance_to_polygon=distance_to_polygon,
        seed=seed,
    )

polygons = {
    7: (566, 137, 567.5, 141),
    8: (569, 137, 571, 141),
    9: (572, 137, 573, 141),
    10: (575, 137, 576.5, 141),
    12: (577.5, 137, 579.5, 141),
    11: (581, 137, 582.5, 141),
    1: (566, 144, 567.5, 147),
    2: (569, 144, 571, 147),
    3: (572.7, 144, 574.5, 147),
    4: (575, 144, 576.5, 147),
    5: (577.5, 144, 579.5, 147),
    6: (581, 144, 582.5, 147),
}
positions = [
    (key, distribute_agents_in_polygon(*coords)) for key, coords in polygons.items()
]

In [ ]:
trajectory_file = "Assignment4.sqlite"
simulation = jps.Simulation(
    model=jps.CollisionFreeSpeedModel(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

exit_area1 = Polygon([(582.5, 142), (582.9, 142), (582.9, 143), (582.5, 143)])
exit_area2 = Polygon([(571.5, 147.7), (572.9, 147.7), (572.9, 147), (571.5, 147)])
exit_id1 = simulation.add_exit_stage(exit_area1)
exit_id2 = simulation.add_exit_stage(exit_area2)

journey_id = simulation.add_journey(jps.JourneyDescription([exit_id1, exit_id2]))

In [ ]:
for room_id, positions in positions:
    for pos in positions:
        if room_id in [1, 2, 3, 4, 7, 8, 9, 10]:
            chosen_exit = exit_id2
        else:
            chosen_exit = exit_id1

        simulation.add_agent(
            jps.CollisionFreeSpeedModelAgentParameters(
                journey_id=journey_id, stage_id=chosen_exit, position=pos
            )
        )

In [ ]:
while simulation.agent_count() > 0 and simulation.iteration_count() < 3000:
    simulation.iterate()

print(f"Simulation time: {simulation.iteration_count()} ")

In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=10)